In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from datetime import datetime as dt
import sys
import boto3
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import YAxis,XAxis,Margin

In [2]:
sys.path.append('..')

In [3]:
from modules.stats import *

In [4]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [5]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
s="nacional"
n="consolidado_nacional"
df=pd.read_sql_table(n, con=cnx,schema=s)

In [7]:
def loglog(df,x,y,z,dpi=150,filename=''):
    df=df.loc[~df[x].isnull(),(x,y,z)]
    df[[x,y]]=df[[x,y]].rolling(7).mean()
    
    xlog_range=10**np.arange(np.log10(df[x].min()),np.log10(df[x].max()),0.3)
    
    fechas=[]
    for xtick in xlog_range:
        #print(xtick)
        idx=(df[x].fillna(0)-xtick).abs().argsort()[:2].iloc[1]
        #print(idx)
        #[1]
        #fechas.append(nacional.loc[idx,'Fecha'])
        fechas.append(idx)
        
    dates=df[z].dt.date.astype(str)
    fechas=dates.iloc[fechas].values
    
    layout = go.Layout(showlegend=False,
        xaxis=XAxis(
            title=x,
            type='log'
        ),
        xaxis2 = XAxis(
            title="Fecha",
            overlaying= 'x', 
            type='log',
            side= 'top',
            tickmode = 'array',
            tickangle=315,
        tickvals = xlog_range,
        ticktext = fechas
        ),
        yaxis=dict(
            title=y,
            type='log'
        ),
    )

    # Create figure with secondary x-axis
    fig = go.Figure(layout=layout)

    # Add traces
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y]),
    )

    fig.add_trace(
        go.Scatter(x=df[x], y=df[y], xaxis='x2'),
    )

    fig.write_html(filename)

In [8]:
loglog(df,'Fallecidos DEIS Acumulado','Fallecidos DEIS Diarios','Fecha',dpi=100,filename=path+'Muertes_DEIS.html')

In [9]:
loglog(df,'Casos Acumulados ICOVID','Casos Nuevos ICOVID','Fecha',dpi=100,filename=path+'Contagios_ICOVID.html')

In [10]:
ACCESS_KEY = os.environ['S3_ACCESS_KEY']
SECRET_KEY = os.environ['S3_SECRET_KEY']
bucket=os.environ['S3_BUCKET']

In [11]:
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)

In [12]:
filelist= [file for file in os.listdir(path) if file.endswith('.html')]

In [13]:
filelist

['Contagios_ICOVID.html', 'Muertes_DEIS.html', 'index.html']

In [14]:
for f in filelist:
    s3.upload_file(path+'/'+f, bucket, f, ExtraArgs={'ACL':'public-read','ContentType':'text/html'})

In [15]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_LOGLOG.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/LOGLOG.ipynb')

0